# Fully Convolutional Interior/Edge Segmentation for 2D Data

---

Classifies each pixel as either Cell Edge, Cell Interior, or Background.

There are 2 different Cell Edge classes (Cell-Cell Boundary and Cell-Background Boundary)

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/TFG/TFG MARINA CALZADA/Repo Github

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TFG/TFG MARINA CALZADA/Repo Github


In [0]:
# !git clone https://github.com/aniramcg/deepcell-tf.git

In [0]:
import os
os.chdir('/content/drive/My Drive/TFG/TFG MARINA CALZADA/Repo Github/deepcell-tf')
!ls

build		    deepcell	LICENSE     requirements-test.txt  setup.py
CODE_OF_CONDUCT.md  Dockerfile	pytest.ini  requirements.txt
CONTRIBUTING.md     docs	README.md   scripts


In [0]:
!python setup.py build_ext --inplace
!pip install -r requirements.txt

running build_ext
skipping 'deepcell/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.6/deepcell/utils/compute_overlap.cpython-36m-x86_64-linux-gnu.so -> deepcell/utils


In [0]:
import os
import errno

import numpy as np
!pip install SimpleITK
import deepcell

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Load the training data

#### Use this command if you are working in Google Colab to avoid RAM out of memory problems


In [0]:
!pip install SimpleITK
import SimpleITK as sitk
# import matplotlib.pyplot as plt

# def load_itk_image(filename):
#     itkimage = sitk.ReadImage(filename)
#     numpyImage = sitk.GetArrayFromImage(itkimage)
#     return numpyImage

path2data = '/content/drive/My Drive/TFG/TFG MARINA CALZADA/clean_data/deepcell/data.npz'
training_data = np.load(path2data)
# X_train = training_data['x_train']
X_test = training_data['x_test']
# y_train = training_data['y_train']
y_test = training_data['y_test']
print(X_test.shape[-1])

# plt.figure()
# plt.subplot(1,2,1)
# plt.imshow(X_train[89,:,:,0])
# plt.subplot(1,2,2)
# plt.imshow(y_test[89,:,:,0])
# plt.show()
# print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))
# print('Xtest.shape: {}\ny.shape: {}'.format(X_test.shape, y_test.shape))


1


In [0]:
path2data = '/content/drive/My Drive/TFG/TFG MARINA CALZADA/clean_data/deepcell/data.npz'
test_size = 0.1 # % of data saved as test
seed = 0 # seed for random train-test split

In [0]:
# test_size = 0.1 # % of data saved as test
# seed = 0 # seed for random train-test split
# from sklearn.model_selection import train_test_split
# training_data = np.load('/content/drive/My Drive/Repo Github/example.npz')
# X = training_data['X']
# y = training_data['y']
# print(y.shape)
# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=test_size, random_state=seed)
# del X, y
# print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))
# plt.figure()
# plt.subplot(1,2,1)
# plt.imshow(X_test[1,:,:,0])
# plt.subplot(1,2,2)
# plt.imshow(y_test[1,:,:,0])
# plt.show()

#### Use the next command if you are running the model in your local computer as google colab crashes completely due to the large size of the data HELA

In [0]:
# # Download the data (saves to ~/.keras/datasets)
# filename = 'HeLa_S3.npz'
# test_size = 0.2 # % of data saved as test
# seed = 0 # seed for random train-test split

# # NOTE: pass the same seed and test_size to load_data and to train_model_conv to train on the same train-test split as the one in the line below
# (X_train, y_train), (X_test, y_test) = deepcell.datasets.hela_s3.load_data(filename, test_size=test_size, seed=seed)

# print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

### Set up filepath constants

In [0]:
# the path to the data file is currently required for `train_model_()` functions

# NOTE: Change DATA_DIR if you are not using `deepcell.datasets`
# DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
DATA_DIR = '/content/drive/My Drive/TFG/TFG MARINA CALZADA/clean_data/deepcell/'
DATA_FILE = '/content/drive/My Drive/TFG/TFG MARINA CALZADA/clean_data/deepcell/'
# filename = 'data.npz'
# DATA_FILE = os.path.join(DATA_DIR, filename)

# confirm the data file is available
# assert os.path.isfile(DATA_FILE)

In [0]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

# ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
ROOT_DIR = '/content/drive/My Drive/Repo Github'
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [0]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'conv_fgbg_model'
conv_model_name = 'conv_edgeseg_model'

n_epoch = 3  # Number of training epochs
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# FC training settings
n_skips = 3  # number of skip-connections (only for FC training)
batch_size = 1  # FC training uses 1 image per batch

# Transformation settings
transform = 'pixelwise'
dilation_radius = 1  # change dilation radius for edge dilation
separate_edge_classes = True  # break edges into cell-background edge, cell-cell edge
n_features = 4 if separate_edge_classes else 3

### First, create a foreground/background separation model

#### Instantiate the fgbg model

In [0]:
from deepcell import model_zoo

fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape = tuple((None, None, 1)),
    # input_shape=tuple((256,256,1)), # indicar el shape de las imágenes de entrenamiento
    last_only=False)

#### Train the model fgbg model

In [0]:
# from deepcell.training import train_model_conv

# fgbg_model = train_model_conv(
#     model=fgbg_model,
#     dataset=DATA_FILE,  # full path to npz file
#     model_name=fgbg_model_name,
#     # test_size=test_size,
#     seed=seed,
#     optimizer=optimizer,
#     n_epoch=n_epoch,
#     batch_size=batch_size,
#     transform='fgbg',
#     model_dir=MODEL_DIR,
#     log_dir=LOG_DIR,
#     lr_sched=lr_sched,
#     rotation_range=180,
#     flip=True,
#     shear=False,
#     zoom_range=(0.8, 1.2))

#### Load the trained fgbg model from the MODEL_DIR

In [0]:
fgbg_model.load_weights(os.path.join(MODEL_DIR, 'fgbg_model' + '.h5'))

### Next, Create a model for the edge/interior segmentation

#### Instantiate the segmentation transform model

In [0]:
# from deepcell import model_zoo

# conv_model = model_zoo.bn_feature_net_skip_2D(
#     fgbg_model=fgbg_model,
#     receptive_field=receptive_field,
#     n_skips=n_skips,
#     n_features=n_features,
#     norm_method=norm_method,
#     n_conv_filters=32,
#     n_dense_filters=128,
#     last_only=False,
#     # input_shape=tuple(X_train.shape[1:]))
#     input_shape=tuple((256,256,1)))

#### Train the segmentation transform model

In [0]:
# from deepcell.training import train_model_conv

# conv_model = train_model_conv(
#     model=conv_model,
#     dataset=DATA_FILE,  # full path to npz file
#     model_name=conv_model_name,
#     test_size=test_size,
#     seed=seed,
#     transform=transform,
#     dilation_radius=dilation_radius,
#     separate_edge_classes=separate_edge_classes,
#     optimizer=optimizer,
#     batch_size=batch_size,
#     n_epoch=n_epoch,
#     log_dir=LOG_DIR,
#     model_dir=MODEL_DIR,
#     lr_sched=lr_sched,
#     rotation_range=180,
#     flip=True,
#     shear=False,
#     zoom_range=(0.8, 1.2),)

### Run the model

#### Make predictions on test data

In [0]:
# test_images = conv_model.predict(X_test)[-1]
test_images_fgbg = fgbg_model.predict(X_test)[-1]
# test_images_fgbg = fgbg_model.evaluate(X_test,batch_size=1,verbose=1)

# print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

#### Post-processing

In [0]:
# threshold the foreground/background
# and remove back ground from edge transform
threshold = 0.9

fg_thresh = test_images_fgbg[..., 1] > threshold
fg_thresh = np.expand_dims(fg_thresh, axis=-1)

test_images_post_fgbg = test_images * fg_thresh
test_images_post_fgbg = fg_thresh


In [0]:
# Label interior predictions
from skimage.measure import label
from skimage import morphology

labeled_images = []
for i in range(test_images_post_fgbg.shape[0]):
    interior = test_images_post_fgbg[i, ..., 2] > .2
    labeled_image = label(interior)
    labeled_image = morphology.remove_small_objects(
        labeled_image, min_size=50, connectivity=1)
    labeled_images.append(labeled_image)
labeled_images = np.array(labeled_images)
labeled_images = np.expand_dims(labeled_images, axis=-1)

print('labeled_images shape:', labeled_images.shape)

In [0]:
import matplotlib.pyplot as plt

index = np.random.randint(low=0, high=X_test.shape[0])
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, ..., 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, ..., 0], cmap='jet')
ax[2].set_title('FGBG Threshold {}%'.format(threshold * 100))

ax[3].imshow(test_images[index, ..., 0] + test_images[index, ..., 1], cmap='jet')
ax[3].set_title('Edge Prediction')

ax[4].imshow(test_images[index, ..., 2], cmap='jet')
ax[4].set_title('Interior Prediction')

ax[5].imshow(labeled_images[index, ..., 0], cmap='jet')
ax[5].set_title('Instance Segmentation')

fig.tight_layout()
plt.show()

In [0]:
import os
import SimpleITK as sitk
import numpy as np

# This function is placed in deepcell.utils.data_utils
# Randomly crop the image to a specific size. For data augmentation
def random_crop(image, label, crop_height, crop_width):
    if (image.shape[0] != label.shape[0]) or (image.shape[1] != label.shape[1]):
        raise Exception('Image and label must have the same dimensions!')
    if (crop_width <= image.shape[1]) and (crop_height <= image.shape[0]):
        pdf_im = np.ones(label.shape) # y is a mask
        pdf_im[label>0]=10000 # pdf aquí es un peso. Por ejemplo 10000. 
        # cropw = int(crop_width/2)
        # croph = int(crop_height/2)
        pdf_im = pdf_im[:-crop_height,:-crop_width] # limit the coordinates in which a centroid can lay
        prob = np.float32(pdf_im)
        prob = prob.ravel()/np.sum(prob) # convert the 2D matrix into a vector and normalize it so you create a distribution of all the possible values between 1 and prod(pdf.shape)(sum=1)
        choices = np.prod(pdf_im.shape) 
        index = np.random.choice(choices, size=1,p = prob) # get a random centroid but following a pdf distribution.
        coordinates = np.unravel_index(index, shape=pdf_im.shape)
        y = coordinates[0][0]
        x = coordinates[1][0]
        return image[y:y+crop_height, x:x+crop_width], label[y:y+crop_height, x:x+crop_width]
    else:
        raise Exception('Crop shape (%d, %d) exceeds image dimensions (%d, %d)!' % (crop_height, crop_width, image.shape[0], image.shape[1]))


def get_data_MARINA(DATAPATH, mode='sample', test_size=.2, seed=0, crop_height=256, crop_width=256):
    """Load data from NPZ file and split into train and test sets

    Args:
        file_name (str): path to NPZ file to load
        mode (str): if 'siamese_daughters', returns lineage information from
            .trk file otherwise, returns the same data that was loaded.
        test_size (float): percent of data to leave as testing holdout
        seed (int): seed number for random train/test split repeatability

    Returns:
        (dict, dict): dict of training data, and a dict of testing data
    """
    # siamese_daughters mode is used to import lineage data
    # and associate it with the appropriate batch
    # if mode == 'siamese_daughters':
    #     training_data = load_trks(file_name)
    #     X = training_data['X']
    #     y = training_data['y']
    #     # `daughters` is of the form:
    #     #
    #     #                   2 children / cell (potentially empty)
    #     #                          ___________|__________
    #     #                         /                      \
    #     #      daughers = [{id_1: [daughter_1, daughter_2], ...}, ]
    #     #                  \___________________________________/
    #     #                                    |
    #     #                       dict of (cell_id -> children)
    #     #
    #     # each batch has a separate (cell_id -> children) dict
    #     daughters = [{cell: fields['daughters']
    #                   for cell, fields in tracks.items()}
    #                  for tracks in training_data['lineages']]

    #     X_train, X_test, y_train, y_test, ln_train, ln_test = train_test_split(
    #         X, y, daughters, test_size=test_size, random_state=seed)

    #     train_dict = {
    #         'X': X_train,
    #         'y': y_train,
    #         'daughters': ln_train
    #     }

    #     test_dict = {
    #         'X': X_test,
    #         'y': y_test,
    #         'daughters': ln_test
    #     }
    #     return train_dict, test_dict

    train_files = os.listdir(os.path.join(DATAPATH, 'train'))
    X_train = None

    for f in train_files:
      input_im = sitk.ReadImage(os.path.join(DATAPATH, 'train',f))
      input_im = sitk.GetArrayFromImage(input_im)
      input_im = input_im[:,:,0]
      mask_im = sitk.ReadImage(os.path.join(DATAPATH, 'train_labels',f))
      mask_im = sitk.GetArrayFromImage(mask_im)
      mask_im = mask_im[:,:,0]
      mask_im[mask_im > 0] = 1
      input_im, mask_im = random_crop(input_im, mask_im, crop_height, crop_width)      
      input_im = input_im.reshape((1, crop_height, crop_width, 1))
      mask_im = mask_im.reshape((1, crop_height, crop_width, 1))
      if X_train is None: 
        X_train = input_im
        y_train = mask_im
      else:
        X_train = np.concatenate((X_train,input_im), axis=1)
        y_train = np.concatenate((y_train,mask_im), axis=1)


      train_files = os.listdir(os.path.join(DATAPATH, 'test'))

    X_test = None
    for f in train_files:
      input_im = sitk.ReadImage(os.path.join(DATAPATH, 'test',f))
      input_im = sitk.GetArrayFromImage(input_im)
      input_im = input_im[:,:,0]
      mask_im = sitk.ReadImage(os.path.join(DATAPATH, 'test_labels',f))
      mask_im = sitk.GetArrayFromImage(mask_im)
      mask_im = mask_im[:,:,0]
      mask_im[mask_im > 0] = 1
      input_im, mask_im = random_crop(input_im, mask_im, crop_height, crop_width)      
      input_im = input_im.reshape((1, crop_height, crop_width, 1))
      mask_im = mask_im.reshape((1, crop_height, crop_width, 1))
      if X_test is None: 
        X_test = input_im
        y_test = mask_im
      else:
        X_test = np.concatenate((X_test,input_im), axis=1)
        y_test = np.concatenate((y_train,mask_im), axis=1)

    train_dict = {
        'X': X_train,
        'y': y_train
    }

    test_dict = {
        'X': X_test,
        'y': y_test
    }

    return train_dict, test_dict
